# REINFORCE Algothm from scratch in PyTorch

The task we are trying to solve is: if you get placed into a random cell within an N by N grid. What is the quickest way to get out of the grid (go to the exit). The exit is fixed and is always in the top right corner.

Add the dependencies:

In [1]:
#!pip install torch
#!pip install tqdm




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm



Set up a simple class to give the grid an interface:

In [3]:
class Grid:
    def __init__(self, n, m, exit_pos, figure_pos):
        self.n = n
        self.m = m
        self.exit_pos = exit_pos
        self.figure_pos = figure_pos

    def move(self, direction):
        x, y = self.figure_pos
        if direction == "up":
            if y < self.n-1:
                self.figure_pos = (x, y+1)
        elif direction == "down":
            if y > 0:
                self.figure_pos = (x, y-1)
        elif direction == "left":
            if x > 0:
                self.figure_pos = (x-1, y)
        elif direction == "right":
            if x < self.m-1:
                self.figure_pos = (x+1, y)

    def is_at_exit(self):
        return self.figure_pos == self.exit_pos

    def get_state(self, device):
        return torch.FloatTensor(self.figure_pos).unsqueeze(0).to(device)
                
    

The first input of the REINFORCE is a policy:


In [4]:
class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 4) # four outputs for the 4 actions

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        x = nn.functional.softmax(x, dim=1)
        return x
        


REINFORCE assumed the episodic task setting. Fortunately for us, in this case the task is clearly episodic: the episode ends whenever the agent finds the exit. In a grid that is small, it happens fairly quickly. We therefore need a function that is able to generate the whole episode

In [5]:
def generate_episode(grid, policy_net, device="cpu", max_episode_len = 100):
    state = grid.get_state(device)
    ep_length = 0
    while not grid.is_at_exit():
        # Convert state to tensor and pass through policy network to get action probabilities
        ep_length+=1
        action_probs = policy_net(state).squeeze()
        log_probs = torch.log(action_probs)
        cpu_action_probs = action_probs.detach().cpu().numpy()
        action = np.random.choice(np.arange(4), p=cpu_action_probs)

        # Take the action and get the new state and reward
        grid.move(actions[action])
        next_state = grid.get_state(device)
        reward = -0.1 if not grid.is_at_exit() else 0

        # Add the state, action, and reward to the episode
        new_episode_sample = (state, action, reward)
        yield new_episode_sample, log_probs

        # We do not want to add the state, action, and reward for reaching the exit position
        if reward == 0:
            break

        # Update the current state
        state = next_state
        if ep_length > max_episode_len:
            return

    # Add the final state, action, and reward for reaching the exit position
    new_episode_sample = (grid.get_state(device), None, 0)
    yield new_episode_sample, log_probs

Remember, policy basically describes probabilities of taking certain actions under certain conditions. Also, remember the REINFORCE update rule needs the gradient of the log of the probability, hence we calculate logs of probabilities here.

We will also need a few helper functions:

In [6]:
def gradients_wrt_params(
    net: torch.nn.Module, loss_tensor: torch.Tensor
):
    # Dictionary to store gradients for each parameter
    # Compute gradients with respect to each parameter
    for name, param in net.named_parameters():
        g = grad(loss_tensor, param, retain_graph=True)[0]
        param.grad = g

def update_params(net: torch.nn.Module, lr: float) -> None:
    # Update parameters for the network
    for name, param in net.named_parameters():
        param.data += lr * param.grad

Then finally code for the algorithm:

In [9]:
policy_net = PolicyNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
policy_net.to(device)

lengths = []
rewards = []

gamma = 0.99
lr_policy_net = 2**-13
optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr_policy_net)

prefix = "reinforce-per-step"

for episode_num in tqdm(range(2500)):
    all_iterations = []
    all_log_probs = []    
    grid = get_good_starting_grid()
    episode = list(generate_episode(grid, policy_net=policy_net, device=device))
    lengths.append(len(episode))
    loss = 0
    for t, ((state, action, reward), log_probs) in enumerate(episode[:-1]):
        gammas_vec = gamma ** (torch.arange(t+1, len(episode))-t-1)
        # Since the reward is -1 for all steps except the last, we can just sum the gammas
        G = - torch.sum(gammas_vec)
        rewards.append(G.item())
        policy_loss = log_probs[action]
        optimizer.zero_grad()
        gradients_wrt_params(policy_net, policy_loss)
        update_params(policy_net, lr_policy_net  * G * gamma**t)




  0%|                                                                                         | 0/2500 [00:00<?, ?it/s]


NameError: name 'get_good_starting_grid' is not defined